# How to match? 

## 0. Import libraries 

In [1]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.matcher import Matcher

warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,10,1,485,True
1,0,10,2,472,True
2,0,10,3,477,True
3,0,10,4,478,True
4,0,10,5,498,True


In [3]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,2,0,False,504.0,421.222222
1,5,0,False,504.0,412.777778
2,6,3,True,471.0,514.444444
3,7,0,False,475.0,437.111111
4,8,0,False,481.5,412.555556
...,...,...,...,...,...
5517,9992,0,False,464.5,424.444444
5518,9993,0,False,469.0,423.888889
5519,9995,0,False,500.0,415.888889
5520,9996,0,False,456.0,425.111111


In [4]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,2,0,0,504.0,421.222222,29,1,2
1,5,0,0,504.0,412.777778,20,0,1
2,6,3,1,471.0,514.444444,48,0,1
3,7,0,0,475.0,437.111111,33,0,1
4,8,0,0,481.5,412.555556,58,1,2


In [5]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [23]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [7]:
# Standard model with base parameters
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results = model.estimate()

Finding index
Finding index
Index(['index', 'user_id', 'signup_month', 'pre_spends', 'age', 'is_male',
       'industry_2', 'index_matched', 'user_id_matched',
       'signup_month_matched', 'pre_spends_matched', 'age_matched',
       'is_male_matched', 'industry_2_matched'],
      dtype='object')
Index(['index', 'user_id', 'signup_month', 'pre_spends', 'age', 'is_male',
       'industry_2', 'index_matched', 'user_id_matched',
       'signup_month_matched', 'pre_spends_matched', 'age_matched',
       'is_male_matched', 'industry_2_matched'],
      dtype='object')


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [8]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [9]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

Finding index
Finding index
Finding index
Finding index
Index(['index', 'user_id', 'signup_month', 'pre_spends', 'age', 'is_male',
       'industry', 'index_matched', 'user_id_matched', 'signup_month_matched',
       'pre_spends_matched', 'age_matched', 'is_male_matched',
       'industry_matched'],
      dtype='object')
Index(['index', 'user_id', 'signup_month', 'pre_spends', 'age', 'is_male',
       'industry', 'index_matched', 'user_id_matched', 'signup_month_matched',
       'pre_spends_matched', 'age_matched', 'is_male_matched',
       'industry_matched'],
      dtype='object')


### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [10]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)

In [11]:
features_importance = model.lama_feature_select()
features_importance

      Feature     Importance
0  pre_spends  208194.750732
1         age  122496.639404
2     is_male   28708.780029


,Feature,Importance
0,pre_spends,208194.750732
1,age,122496.639404
2,is_male,28708.780029


In [12]:
features = features_importance['Feature'].to_list()

In [13]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

Finding index
Finding index
Finding index
Finding index
Index(['index', 'user_id', 'signup_month', 'pre_spends', 'age', 'is_male',
       'industry', 'index_matched', 'user_id_matched', 'signup_month_matched',
       'pre_spends_matched', 'age_matched', 'is_male_matched',
       'industry_matched'],
      dtype='object')
Index(['index', 'user_id', 'signup_month', 'pre_spends', 'age', 'is_male',
       'industry', 'index_matched', 'user_id_matched', 'signup_month_matched',
       'pre_spends_matched', 'age_matched', 'is_male_matched',
       'industry_matched'],
      dtype='object')


In [14]:
model.matcher.df_matched

,index,user_id,signup_month,pre_spends,age,is_male,industry,index_matched,user_id_matched,signup_month_matched,pre_spends_matched,age_matched,is_male_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,2271,4082,3,514.0,49,0,1,3265,5884,0,504.0,19,1,1,407.555556,516.555556,513.990788,0,1
1,2270,4079,3,491.0,50,1,1,3731,6719,0,495.0,56,1,1,415.000000,515.888889,515.979098,0,1
2,4329,7881,3,486.0,28,1,1,850,1544,0,492.0,59,0,1,413.333333,531.000000,533.318479,0,1
3,4323,7872,3,531.0,26,1,1,3713,6685,0,528.5,44,0,1,425.666667,521.444444,521.324932,0,1
4,2364,4231,3,498.5,60,0,1,4943,8941,0,489.5,25,1,1,422.222222,532.888889,535.160870,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5517,2442,4365,0,472.0,63,0,2,2599,4658,3,511.5,57,0,2,531.888889,427.000000,426.287163,1,0
5518,2444,4367,0,489.0,43,1,2,1037,1880,3,481.5,29,0,2,518.666667,427.666667,427.590970,1,0
5519,2448,4375,0,466.0,48,0,2,4619,8371,3,501.0,57,1,2,516.888889,414.555556,415.176078,1,0
5520,2453,4388,0,482.0,65,0,2,2388,4271,3,507.5,58,0,2,537.222222,423.333333,422.847975,1,0


## 3. Results  
### 3.1 ATE, ATT, ATC

In [15]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,99.451855,1.061501,0.0,97.371313,101.532397
ATC,99.430147,1.128720,0.0,97.217855,101.642438
ATT,99.704461,0.961992,0.0,97.818957,101.589965


### 3.2 SMD, PSI, KS-test, repeats

In [16]:
# matching quality result - SMD
model.quality_result['smd']

,match_control_to_treat,match_treat_to_control
pre_spends,0.061878,0.077701
age,0.083748,0.030465
is_male,0.027445,0.000787


In [17]:
# matching quality result - PSI
model.quality_result['psi']

,column,anomaly_score,check_result,column,anomaly_score,check_result
0,pre_spends_treated,0.06,OK,pre_spends_untreated,0.09,OK
1,age_treated,3.32,NOK,age_untreated,1.70,NOK
2,is_male_treated,0.00,OK,is_male_untreated,0.00,OK
3,post_spends_treated,16.11,NOK,post_spends_untreated,8.24,NOK


In [18]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
pre_spends,1.654789e-01,8.230024e-08
age,1.863776e-09,1.795862e-72
is_male,1.000000e+00,1.000000e+00


In [19]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.56, 'match_treat_to_control': 0.03}

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [20]:
 
model.validate_result()
 

Finding index
Finding index
Finding index
Finding index
Index(['index', 'user_id', 'signup_month', 'pre_spends', 'age', 'is_male',
       'industry', 'index_matched', 'user_id_matched', 'signup_month_matched',
       'pre_spends_matched', 'age_matched', 'is_male_matched',
       'industry_matched'],
      dtype='object')
Index(['index', 'user_id', 'signup_month', 'pre_spends', 'age', 'is_male',
       'industry', 'index_matched', 'user_id_matched', 'signup_month_matched',
       'pre_spends_matched', 'age_matched', 'is_male_matched',
       'industry_matched'],
      dtype='object')
Finding index
Finding index
Finding index
Finding index
Index(['index', 'user_id', 'signup_month', 'pre_spends', 'age', 'is_male',
       'industry', 'index_matched', 'user_id_matched', 'signup_month_matched',
       'pre_spends_matched', 'age_matched', 'is_male_matched',
       'industry_matched'],
      dtype='object')
Index(['index', 'user_id', 'signup_month', 'pre_spends', 'age', 'is_male',
       'indu

{'post_spends': 0.10069467972987507}

In [22]:
model.matcher.df_matched

,index,user_id,signup_month,pre_spends,age,is_male,industry,index_matched,user_id_matched,signup_month_matched,pre_spends_matched,age_matched,is_male_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,2277,4089,0,474.0,18,1,1,4555,8264,0,500.5,23,1,1,407.555556,416.000000,415.566611,0,1
1,2279,4091,0,474.5,35,1,1,3085,5547,0,497.5,26,1,1,415.000000,425.444444,424.599400,0,1
2,2249,4041,0,490.5,37,1,1,321,589,0,481.0,25,1,1,425.666667,420.888889,421.923572,0,1
3,2255,4052,0,461.0,47,1,1,5293,9581,3,507.5,31,0,1,433.444444,428.555556,430.503430,0,1
4,2343,4202,0,478.5,44,1,1,4979,8995,0,490.5,28,0,1,425.333333,411.222222,410.672560,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5517,2442,4365,0,472.0,63,0,2,3769,6795,0,503.0,65,0,2,429.555556,421.444444,421.606732,1,0
5518,2444,4367,0,489.0,43,1,2,1287,2322,0,474.5,46,1,2,418.444444,432.777778,431.888806,1,0
5519,2448,4375,0,466.0,48,0,2,4652,8429,0,492.0,56,0,2,411.000000,405.333333,405.341447,1,0
5520,2453,4388,0,482.0,65,0,2,3769,6795,0,503.0,65,0,2,409.444444,411.111111,411.365731,1,0
